# Time series tools package

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from bokeh.plotting import figure
from bokeh.layouts import gridplot
from bokeh.io import output_file, output_notebook, show
import warnings
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
data = pd.read_csv("data/walmart_stock.csv", index_col="Date", parse_dates=True)

In [ ]:
data.head()

In [ ]:
open = data['Open']

In [ ]:
open.shape

# Plotting the data

In [ ]:
from mltools.plottingTool.mltools_plot import time_series_plot

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6)
show(f2)

# Reconstruct Time Series

In [ ]:
from mltools.timeSeriesTools import imputeTS

Eliminiamo una buona "fetta" di dati e vediamo come gestire un serie temporali con diversi "buchi", di dimensione variabile: 

In [ ]:
#togliamo una "fetta" di dati
partial_df = open[~open.index.isin(pd.date_range(start = '2014-01-09', end='2014-04-12', freq='D'))]

In [ ]:
partial_df.shape

Per ricostruire il dataset completo, utilizziamo la funzione di pandas per ricostruire il range di date, che saranno poi utilizzate come indici per il nuovo dataset:

In [ ]:
complete_data = imputeTS.ReconstructTS(data=partial_df, start = open.index[0], end = open.index[-1], freq='D')

In [ ]:
complete_data.shape

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = complete_data, fig=f2, alpha = 0.6)
show(f2)

È anche possibile ricostruire la serie utilizzando una interpolazione lineare. Questo è possibile impostante il flag "interpolate" uguale a True e, inoltre, scegliamo anche di limitare i NaN consecutivi, per cui è accettabile l'interpolazione, a 5:

In [ ]:
complete_data = imputeTS.ReconstructTS(data=partial_df, start = open.index[0], end = open.index[-1], freq='D', 
                                       interpolate=True, window=5)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = complete_data, fig=f2, alpha = 0.6)
show(f2)

# Smoothing filters

### Smoothing by resample the time series 

In [ ]:
from mltools.timeSeriesTools import smoothingFilters

In [ ]:
mean_5day = smoothingFilters.AggregateData(method="mean", interval="5D")
max_5day = smoothingFilters.AggregateData(method="max", interval="5D")
min_5day = smoothingFilters.AggregateData(method="min", interval="5D")
perc_5day = smoothingFilters.AggregateData(method="percentile", interval="5D", percentile=0.25)

In [ ]:
open_mean_5day = mean_5day.fit(open)
open_max_5day = max_5day.fit(open)
open_min_5day = min_5day.fit(open)
open_perc_5day = perc_5day.fit(open)

In [ ]:
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6)
f2 = time_series_plot.plot_time_series(data = open_mean_5day, fig=f2, color = "red", linestyle="dashed", legend="mean")
f2 = time_series_plot.plot_time_series(data = open_max_5day, fig=f2, color = "green", linestyle="solid", legend = "max")
f2 = time_series_plot.plot_time_series(data = open_min_5day, fig=f2, color = "orange", linestyle="solid", legend = "min")
f2 = time_series_plot.plot_time_series(data = open_perc_5day, fig=f2, color = "lightblue", linestyle="dashed", legend = "percentile")
show(f2)

### Moving average

We can use a simple moving average or an exponential moving average:

In [ ]:
simple_ma = smoothingFilters.MovingAverage(method="simple", window_size=5)
exp_ma = smoothingFilters.MovingAverage(method="exponential", window_size=8)

In [ ]:
simple_ma_data = simple_ma.fit(open)
exp_ma_data = exp_ma.fit(open)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "green", linewidth=2)
f2 = time_series_plot.plot_time_series(data = simple_ma_data, fig=f2, alpha = 0.6, color = "blue", legend = "simple MA")
f2 = time_series_plot.plot_time_series(data = exp_ma_data, fig=f2, alpha = 0.6, color = "red", legend = "exponential MA")
show(f2)

### Savitzky Golay filter

In [ ]:
savgol = smoothingFilters.SavGol_smoothing(polyorder=1, window_size=5)

In [ ]:
savgol_data = savgol.fit(open)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "green", linewidth=2)
f2 = time_series_plot.plot_time_series(data = savgol_data, fig=f2, alpha = 0.6, color = "blue", legend = "Sav-Gol Filter")
show(f2)

# Trend analysis

In [ ]:
from mltools.timeSeriesTools import trendAnalysis

### Linear Trend

In [ ]:
linear_regression = trendAnalysis.TrendAnalysis(method="linear")

In [ ]:
linear_trend = linear_regression.fit(open)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "blue", linewidth=1)
f2 = time_series_plot.plot_time_series(data = linear_trend, linewidth=3, fig=f2, alpha = 0.6, color = "red", legend = "Linear trend")
show(f2)

### Lowess Trend

In [ ]:
lowess_regression = trendAnalysis.TrendAnalysis(method="lowess", smooth=0.2)

In [ ]:
lowess_trend = lowess_regression.fit(open)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "blue", linewidth=1)
f2 = time_series_plot.plot_time_series(data = lowess_trend, linewidth=2, fig=f2, alpha = 0.6, color = "red", legend = "Lowess trend")
show(f2)

### Expanding mean

In [ ]:
expanding_mean = trendAnalysis.TrendAnalysis(method="expanding_mean", period=1)

In [ ]:
expanding_trend = expanding_mean.fit(open)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "blue", linewidth=1)
f2 = time_series_plot.plot_time_series(data = expanding_trend, linewidth=2, fig=f2, alpha = 0.6, color = "red", legend = "Lowess trend")
show(f2)

### Bollinger bands

In [ ]:
bollinger_bands = trendAnalysis.TrendAnalysis(method="bollinger_bands", windows_size=20, delta=2)

In [ ]:
upper_band, lower_band = bollinger_bands.fit(open) 

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.6, color = "blue", linewidth=1)
f2 = time_series_plot.plot_time_series(data = upper_band, linewidth=1, fig=f2, alpha = 0.6, color = "red", legend = "Upper band")
f2 = time_series_plot.plot_time_series(data = lower_band, linewidth=1, fig=f2, alpha = 0.6, color = "green", legend = "Lower band")
show(f2)

# ETS Decomposition

In [ ]:
from mltools.timeSeriesTools import ets

In [ ]:
ets.ETS_decomposition.methods_avaible

In [ ]:
airline_passanger = pd.read_csv("data/airline_passengers.csv", index_col="Month")

In [ ]:
airline_passanger.tail()

In [ ]:
airline_passanger.dropna(inplace=True)

In [ ]:
airline_passanger.index = airline_passanger.index.astype("datetime64[ns]")

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = airline_passanger, fig=f2, alpha = 0.6)
show(f2)

### Hodrick-Prescott filter

In [ ]:
hp_filter = ets.ETS_decomposition(method="hp_filter")

In [ ]:
cycle, trend = hp_filter.fit(airline_passanger["Thousands of Passengers"])

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = airline_passanger["Thousands of Passengers"], fig=f2, alpha = 0.6, color = "blue", linewidth=1)
f2 = time_series_plot.plot_time_series(data = trend, linewidth=2, fig=f2, alpha = 0.6, color = "red", legend = "Trend")
f2 = time_series_plot.plot_time_series(data = cycle, linewidth=2, fig=f2, alpha = 0.6, color = "orange", legend = "Cycle")
show(f2)

### SSA Decomposition

In [ ]:
ssa_L5 = ets.ETS_decomposition(method="ssa", L = 5)

In [ ]:
F_ssa_L5 = ssa_L5.fit(open)

In [ ]:
ssa_comp = F_ssa_L5.components_to_df()

In [ ]:
ssa_comp.head()

In [ ]:
F_ssa_L5.plot_wcorr()

In [ ]:
F0 = F_ssa_L5.reconstruct(0)
F1 = F_ssa_L5.reconstruct([1,2,3,4])

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = open, fig=f2, alpha = 0.4, color = "blue", linewidth=1.2, legend = "Original data")
f2 = time_series_plot.plot_time_series(data = F0, linewidth=1, fig=f2, alpha = 0.8, color = "red", legend = "F0")
f2 = time_series_plot.plot_time_series(data = F1, linewidth=1, fig=f2, alpha = 0.8, color = "green", legend = "F1")
show(f2)

In [ ]:
ssa_L20 = ets.ETS_decomposition(method="ssa", L=20)

In [ ]:
F_ssa_L20 = ssa_L20.fit(airline_passanger["Thousands of Passengers"])
ssa_comp = F_ssa_L20.components_to_df()

In [ ]:
F_ssa_L20.plot_wcorr()

In [ ]:
F0 = F_ssa_L20.reconstruct(0)
F1 = F_ssa_L20.reconstruct([1,2])
F2 = F_ssa_L20.reconstruct([3,4])
F3 = F_ssa_L20.reconstruct(slice(5,12))
F4 = F_ssa_L20.reconstruct(slice(12,20))

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = F0, linewidth=1, fig=f2, alpha = 0.8, color = "blue", legend = "F0")
f2 = time_series_plot.plot_time_series(data = F1, linewidth=1, fig=f2, alpha = 0.8, color = "green", legend = "F1")
f2 = time_series_plot.plot_time_series(data = F2, linewidth=1, fig=f2, alpha = 0.8, color = "red", legend = "F2")
show(f2)

In [ ]:
F_ssa_L20_main_components = F_ssa_L20.get_main_components(corr_threshold=0.45,adjust=0)
F_ssa_L20_main_components.head()

In [ ]:
F0 = F_ssa_L20_main_components.iloc[:,0]
F1 = F_ssa_L20_main_components.iloc[:,1]
F2 = F_ssa_L20_main_components.iloc[:,2]
noise = F_ssa_L20_main_components.iloc[:,-1]

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = F0, linewidth=1, fig=f2, alpha = 0.8, color = "blue", legend = "F0")
f2 = time_series_plot.plot_time_series(data = F1, linewidth=1, fig=f2, alpha = 0.8, color = "green", legend = "F1")
f2 = time_series_plot.plot_time_series(data = F2, linewidth=1, fig=f2, alpha = 0.8, color = "red", legend = "F2")
f2 = time_series_plot.plot_time_series(data = noise, linewidth=1, fig=f2, alpha = 0.8, color = "purple", legend = "Noise")
show(f2)

In [ ]:
print(F_ssa_L20.groups)

## Seasonal Decomposition

In [ ]:
sd = ets.ETS_decomposition(method="seasonal_decomposition")

In [ ]:
data = airline_passanger["Thousands of Passengers"]

In [ ]:
trend, seasonal, residual = sd.fit(data)

In [ ]:
output_notebook()
f1 = figure(x_axis_type="datetime")
f1 = time_series_plot.plot_time_series(data = data, title='Airline Passengers',
                                       linewidth=1, fig=f1, alpha = 0.8)
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = trend, title='Airline Passengers - Trend',
                                       linewidth=1, fig=f2, alpha = 0.8)

f3 = figure(x_axis_type="datetime")
f3 = time_series_plot.plot_time_series(data = seasonal, title='Airline Passengers - Stagionalità',
                                       linewidth=1, fig=f3, alpha = 0.8)

f4 = figure(x_axis_type="datetime")
f4 = time_series_plot.plot_time_series(data = residual, title='Airline Passengers - Residui',
                                       linewidth=1, fig=f4, alpha = 0.8)

f = gridplot([f1], [f2], [f3], [f4])
show(f)

# Change point analysis

In [ ]:
from mltools.timeSeriesTools import CPA

In [ ]:
# Possiamo vedere quali metodi usare con l'attributo 'methods_av'
CPA.methods_av

Il pacchetto utilizza delle librerie R che sono state incluse all'interno di un wrapper Python. Per questo motivo si consiglia, prima di utilizzare le funzioni messe a disposizione, di utilizzare la funzione get_libraries per identificare le librerie R necessarie. I pacchetti che non vengono individuati possono essere installati direttamente attraverso le caselle di input visualizzate e seguendo la procedura guidata. 

In [ ]:
#POSSIAMO INSTALLARE LE LIBRERIE NECESSARIE CON LA FUNZIONE "get_libraries"
#NB: Prophet richiede molto tempo per l'installazione
#CPA.get_libraries()

In [ ]:
df = pd.read_csv("data/DJI.csv", index_col='Date', parse_dates=True)
df.head()

La ricerca dei changepoints viene fatta istanziando un oggetto CPA e scegliendo il metodo che si vuole utilizzare. Richiamando il metodo _fit()_ si effettua la ricerca dei cp e il risultato è un dizionario che ha come chiave il nome del modello e come valore la lista degli indici dove sono stati trovati i changepoints.

* **Changepoints per la media**

In [ ]:
wbs = CPA(method = 'wbs') 
cp_wbs = wbs.fit(data = df['Close'], model_param={'n_checkpoints':5,
                                                  'penalty_wbs':'SSIC',
                                                  'n_intervals':400,
                                                  'th_wbs':0.3})

In [ ]:
amoc = CPA(method='AMOC')
cp_amoc = amoc.fit(data = df['Close'], model_param={'n_checkpoints':5,
                                                    'penalty_bs':'AIC'})

In [ ]:
binseg = CPA(method='BinSeg')
cp_binseg = binseg.fit(data = df['Close'], model_param={'n_checkpoints':5,
                                                    'penalty_bs':'AIC'})

In [ ]:
segneigh = CPA(method='SegNeigh')
cp_segneigh = segneigh.fit(data = df['Close'], model_param={'n_checkpoints':5,
                                                    'penalty_bs':'AIC'})

In [ ]:
%time bcp = CPA(method='bcp')
cp_bcp = bcp.fit(data = df['Close'], model_param={'n_checkpoints':5,
                                                    'penalty_bs':'AIC'})

In [ ]:
prophet = CPA(method='PROPHET')
cp_prophet = prophet.fit(data = df['Close'])

In [ ]:
%time ecp = CPA(method='ecp')
cp_ecp = ecp.fit(data = df['Close'])

In [ ]:
CPA.plot_changepoints(data = df['Close'],
                      cp_list=[cp_wbs, cp_amoc, cp_binseg,
                               cp_segneigh, cp_bcp, cp_ecp],
                      colors='#123456')

* **Changepoints per la varianza**

In [ ]:
amoc = CPA(method='AMOC')
#NB: AMOC ritorna sempre ad un unico cp
cp_amoc = amoc.fit(data = df['Close'],
                   compute_mean = False,
                   model_param={'n_checkpoints':5,
                                'penalty_bs':'AIC'})

In [ ]:
binseg = CPA(method='BinSeg')
cp_binseg = binseg.fit(data = df['Close'],
                       compute_mean = False,
                       model_param={'n_checkpoints':5,
                                    'penalty_bs':'AIC'})

In [ ]:
segneigh = CPA(method='SegNeigh')
cp_segneigh = segneigh.fit(data = df['Close'],
                           compute_mean=False,
                           model_param={'n_checkpoints':5,
                                        'penalty_bs':'AIC'})

In [ ]:
pelt = CPA(method='PELT')
cp_pelt = pelt.fit(data = df['Close'],
                   compute_mean=False,
                   model_param={'n_checkpoints':5,
                                'penalty_bs':'AIC'})

In [ ]:
CPA.plot_changepoints(data = df['Close'], cp_list=[cp_amoc, cp_binseg, cp_segneigh, cp_pelt], colors='#123456')

# Forecasting

In [ ]:
from mltools.timeSeriesTools import forecasting

In [ ]:
# Time series log transformation 
data = pd.read_csv("data/airline_passengers.csv", index_col="Month")
data.dropna(inplace=True)
data.index = data.index.astype("datetime64[ns]")
data.columns = ['Passengers']
data.index.name = 'Months'
ts_log = np.log(data)

In [ ]:
ts_log.head()

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = data["Passengers"], fig=f2, alpha = 0.6,
                                       color = "blue", linewidth=1)
show(f2)

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = ts_log["Passengers"], linewidth=1, fig=f2, alpha = 0.6, color = "red")
show(f2)

### Verifica di stazionarietà della Time Series

In [ ]:
from statsmodels.tsa.stattools import adfuller

result = adfuller(ts_log["Passengers"])

print('Original data')
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('  %s: %.3f' % (key, value))

In [ ]:
# Con l'attributo di classe models_available è possibile richiamare la lista dei modelli
# implementati dalla classe forecasting

forecasting.ForecastingTS.models_available

Il metodo _plot_diagram_ permette di eseguire una prima ispezione, plottando, oltre alla time series, la distribuzione dei suoi valori e i diagrammi di Autocorrelazione totale e parziale

In [ ]:
forecasting.plot_diagram(ts_log['Passengers'], bins=10,lags=50)

* **ARIMA**

Come prima cosa vediamo come fittare un modello SARIMAX sui dati a disposizione. Per fare questo, si parte instanziando il modello come istanza della classe ForecastingTS. Con la stessa classe di metodi è possibile ottenere anche un normale modello ARIMA: in questo caso è necessario settare ulteriori parametri passandoli attraverso il dizionario model_param. Per ottenere un modello arima è sufficiente settare la variabile "exog" come None (default).

In [ ]:
# Fitting Arima model
arima = forecasting.ForecastingTS(model='SARIMAX')

Il fit del metodo è identico a quanto visto in precedenza: è sufficiente passare una series il cui indice è la data dell'osservazione. Oltre questo valore, è possibile specificare, nel dizionario fit_params, altri parametri che verranno utilizzati dalla funzione che effettua il fitting del modello  come "s" (periodicità) e "max_value" (limite massimo in cui far variare i parametri durante il gridsearch).

In [ ]:
%time res_arima = arima.fit(ts_log['Passengers'], gridsearch=True, fit_params={'s': 12, 'max_value': 2})

In [ ]:
res_arima

In [ ]:
pred_arima = arima.predict(start_date= '1960-12', end_date='1965-12')

* **HWES (Holt-Winter's Exponential Smoothing)**

Nel secondo caso utilizziamo lo smoothing esponenziale per effettuare le nostre previsioni. Con questo metodo è possibile analizzare serie univariate con componenti di trend/stagionalità. 

In [ ]:
hwse = forecasting.ForecastingTS(model='HWES', model_params={'seasonal_periods': 12})

In [ ]:
%time res_hwes = hwse.fit(ts_log['Passengers'], gridsearch=True)

In [ ]:
res_hwes

In [ ]:
pred_hwes = hwse.predict(start_date= '1960-12-01', end_date='1965-12-01')

* **Facebook Prophet**



In [ ]:
# Fitting prophet model 
prophet = forecasting.ForecastingTS(model='Prophet')
#per il modello di crescita logistica (limitata)
#prophet = forecasting.ForecastingTS(model='Prophet', gridsearch=True, model_params={'growth': 'logistic'})

In [ ]:
cap = pd.Series(np.repeat(ts_log['Passengers'].quantile(), repeats=len(ts_log['Passengers'])))

In [ ]:
%time res_prophet = prophet.fit(ts_log['Passengers'], gridsearch=True)
#per il modello di crescita logistica (limitata)
#res_prophet = prophet.fit(ts_log['Passengers'], fit_params = {'cap':cap})

In [ ]:
res_prophet

In [ ]:
pred_prophet = prophet.predict(start_date= '1960-12-01',
                               end_date='1965-12-01',
                               pred_params={'freq': 'MS'})

### Analisi dei residui sul training set

In [ ]:
forecasting.plot_diagnostics(hwse.residuals)

### Confronto delle previsioni

In [ ]:
output_notebook()
f2 = figure(x_axis_type="datetime")
f2 = time_series_plot.plot_time_series(data = ts_log, linewidth=1, fig=f2,
                                       alpha = 0.8, color = "blue", legend = "Passengers")
f2 = time_series_plot.plot_time_series(data = pred_arima.predicted_mean, linewidth=1, fig=f2,
                                       alpha = 0.8, color = "green", legend = "SARIMAX Forecastings")
f2 = time_series_plot.plot_time_series(data = pred_hwes, linewidth=1, fig=f2,
                                       alpha = 0.8, color = "red", legend = "HWES Forecastings")
f2 = time_series_plot.plot_time_series(data = pred_prophet, linewidth=1, fig=f2,
                                       alpha = 0.8, color = "orange", legend = "PROPHET Forecastings")
show(f2)

# Anomaly detection

# Clustering 